<a href="https://colab.research.google.com/github/alexayanar/colab/blob/master/Pandas_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pandas Coding Challenge/Refresher from FreeCodeCamp.org.

In this challenge, I had to use the given CSV that contained information of tens of thousands of workers, with information such as age, education, race, sex, income, etc. The full list can be seen a few blocks down when I used df.head(). Using Pandas, I manipulated the data to filter, calculate, and obtain whatever data was needed. 

In [79]:
import pandas as pd
import numpy as np

The CSV is from a 1994 Census, according to FreeCodeCamp.org

In [80]:
df = pd.read_csv('/content/adult.data.csv')

As a way to understand just exactly what data and columns I have to work with, I make it a point to continually check on the data itself. Otherwise, I might find a more efficient approach. 

In [81]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


The first question of the challenge was to find the how many people of each race are represented in the dataframe. Using value_counts(), the information is neatly listed showing that the majority of those surveyed are White people. This indicates a clear bias in sampling, evident by the unproportinate ratio between races, as well as the many groups unrepresented. 

In [82]:
df.race.value_counts()

White                 27816
Black                  3124
Asian-Pac-Islander     1039
Amer-Indian-Eskimo      311
Other                   271
Name: race, dtype: int64

The second question asked me to find the average age of men. To make this easier on myself, I selected the sex and age columns and made another dataframe, which I then filtered by sex. Finally, I was able to use .mean() to find the average age of men, which is 39. 

In [83]:
men_age = df[['sex', 'age']]
men = men_age[men_age['sex'] == 'Male']

men.mean()

age    39.433547
dtype: float64

Next, I found the percentage of people who have a Bachelor's degree. I interpreted this question as how many people's highest degree is a Bachelor's. First, I grouped the dataframe by education and counted how many of each were found. The second action is to make the dataframe more readable. To obtain the final number, I divided the number of people with a Bachelor's by the total sum of people represented in the dataframe. 16.45% of the people in the dataframe have a Bachelors.

In [84]:
df2 = df.groupby(by='education').agg('count')
df2 = df2['education-num']
percentage = df2['Bachelors']/df2.sum()

print(percentage)


0.16446055096587942


To find the percentage of people who have a higher education (defined as Bachelor's, Master's, or Doctorate) and earn more than $50K, I had a few options. My first approach was to filter the dataframe with one educational level, then append the other educational levels. Unfortunately, that approach was both unsophisticated and faulty. After studying df.head() and the columns represented, I finally realized that I could use 'educational-num' in my favor, as the target population will have studied at least 13 years. Thus, I changed my approach to first filter the dataframe into two columns: 'education-num' and 'salary.'

In [85]:
ed_sal = df[['education-num', 'salary']]

Then, I filtered the rows to only show people who had earned at least a Bachelor's degree, which takes 13 years.

In [86]:
ed_sal = ed_sal[ed_sal['education-num'] >= 13]

Finally, I grouped the dataframe by salary: those who earned over 50K and those who earned less than 50K. The first function shows how many people earned more than 50K: 3909. The second function shows how many people are represented in total, those with advanced educations. The percentage of people with advanced education and earn over $50K is 48.46%. 

In [87]:
ed_sal.groupby(by = 'salary').agg('count') # 3909
ed_sal.count() #8067

rich_educated = 3909/8067

print(rich_educated)

0.4845667534399405


What about those that earned over 50K without advanced education? This process was similar to the previous question, with the most notable distinction being in the function, changing '>= 13' to '< 13.' The percentage of people without advanced degrees who earn over 50K is 16.05%. 

In [88]:
ed1 = df[['education-num', 'salary']]
ed1 = ed1[ed1['education-num'] < 13]
ed1 = ed1.groupby(by = 'salary').agg('count') #3932
ed1.sum() #24494
rich_uned = 3932/24494
print(rich_uned)

0.16052910916959257


To find the minimum amount of hours a person works a week, I used the .min() function on the column 'hours-per-week.' That column tracks the amount of hours each individual works a week. Surprisingly, the person who works the least in the whole dataframe works only 1 hour a week. 

In [89]:
df['hours-per-week'].min()

1

What percentage of the people who work the minimum number of hours per week have a salary of more than 50K? Assuming that the minimum number of hours per week is 20, the answer is 7%. If we assume the minium number of hours is 40, then it changes to 17.27%.

In [90]:
min_num = df[['hours-per-week', 'salary']]
min_num1 = min_num[min_num['hours-per-week'] <= 20]
min_num1.groupby(by = 'salary').agg('count') #60 at 20 min

min_num2 = min_num[min_num['hours-per-week'] <= 40]
min_num2.groupby(by = 'salary').agg('count') #1179 at 40 min


,hours-per-week
salary,
<=50K,18995
>50K,3985


In [91]:
min_num1.count() #853
min_num2.count() #6827

twen_min = 60/853
forty_min = 1179/6827

The trickiest question in the whole problem set was finding the country with the highest percentage of people who earned over 50K. I had a few unsuccessful approaches before finally settling on creating two seperate series: one with count of people who earn over 50K, and one with the population count of each country. Highest creates the dataframe that contains only the country and the salary. Total contains the population count. 

In [92]:
highest = df[['native-country', 'salary']]
total = highest.groupby(by = 'native-country').agg('count')
total

,salary
native-country,
?,583
Cambodia,19
Canada,121
China,75
Columbia,59
Cuba,95
Dominican-Republic,70
Ecuador,28
El-Salvador,106


High filters the previous dataframe to contain only those who earn more than 50K. 

In [93]:
high = highest[highest['salary'] == '>50K']
high

,native-country,salary
7,United-States,>50K
8,United-States,>50K
9,United-States,>50K
10,United-States,>50K
11,India,>50K
...,...,...
32539,United-States,>50K
32545,United-States,>50K
32554,United-States,>50K
32557,United-States,>50K


Then, I group High into countries and then count how many people represent each country. 

In [94]:
high1 = high.groupby(by = 'native-country').agg('count')

Finally, high_percent is a series which holds the final calculations, using the count of 50K+ earner counts over the population count. 

In [95]:
high_percent = high1['salary']/total['salary']

Using .max() lets us find the maximum percentage in the high_percent series, which is France at 41.86%. 

In [96]:
high_percent.max()

0.4186046511627907

The last problem in the problem set asks the most popular occupation in India among those who earn over 50K. To solve this problem, I filter a data set multiple times. First, to create a subset that contains only country, occupation, and salary. Then, I filter the dataset to only contain Indian workers. With that done, the last filter is to include workers who earn over 50K. With that, we can finally group by occupation and count each category, leaving us with the answer. The most popular occupation among the target group is 'prof-speciality.'

In [97]:
popular = df[['native-country', 'occupation', 'salary']]
popular = popular[popular['native-country'] == 'India']
popular = popular[popular['salary'] == '>50K']
popular = popular.groupby(by = 'occupation').agg('count')
popular

,native-country,salary
occupation,,
Adm-clerical,1,1
Exec-managerial,8,8
Other-service,2,2
Prof-specialty,25,25
Sales,1,1
Tech-support,2,2
Transport-moving,1,1


Whew!